In [ ]:
%run helper_functions.ipynb

PATH = 'final_community_df.csv'
DIRECTORY_PATH = "/Users/jungh/Desktop/great_expectations/docs/docusaurus/docs/guides/"

In [ ]:
all_file_mapping = process_md_files(DIRECTORY_PATH)

In [ ]:

# Get the document_objects
document_objects = extract_document_objects(all_file_mapping)

openai_api_key = os.environ.get('OPENAI_API_KEY')
print(openai_api_key)
# Create the vectorstore
vectorstore = Chroma.from_documents(documents=document_objects, embedding=OpenAIEmbeddings())

In [ ]:
# Define the function to handle the question submission and display the results
def on_question_submit(b):
    question = text_box.value
    score_threshold = float(slider.value)

    retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})

    # Get the retrieved_docs
    retrieved_docs = retriever.get_relevant_documents(question)

    # Clear the previous output
    output.clear_output()

    # Check if there are relevant documents or not and display the output
    with output:
        if len(retrieved_docs) == 0:
            print("No relevant documents found.")
        else:
            print(retrieved_docs)

# Define the function to handle the "Generate Response" button click and display the result
def generate_response(b):
    question = text_box.value
    score_threshold = float(slider.value)

    retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})

    retrieved_docs = retriever.get_relevant_documents(question)

    # Clear the previous output
    output.clear_output()

    # Check if there are relevant documents or not and display the output
    with output:
        if len(retrieved_docs) == 0:
            print(f"No relevant documents found with the query: {question}")
        else:        
            # Create the ChatOpenAI model and the RetrievalQA chain
            llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
            qa_chain = RetrievalQA.from_chain_type(llm, retriever= retriever, return_source_documents=True)    
            result = qa_chain({"query": question})
            # Display the result    
            print(result['result'])
            
# Create the widgets
text_box = widgets.Text(
    description='Enter your question:',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)

slider = widgets.FloatSlider(
    value=0.7,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Score Threshold:',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)

submit_button = widgets.Button(description='Source Document', )
submit_button.on_click(on_question_submit)

generate_button = widgets.Button(description='Generate Response')
generate_button.on_click(generate_response)

# Create an HBox to place the buttons side by side
buttons_box = widgets.HBox([submit_button, generate_button])

# Create the output widget for displaying the results
output = widgets.Output(
    layout=widgets.Layout(width='90%', height='300px', border='1px solid gray', overflow = 'auto')
)

# Display the widgets and output
display(text_box)
display(slider)
display(buttons_box) 
display(output)